# 4. Porto Simple Stacker LB 0.284

## Loading Packages and Dataset

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score

from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier, RandomForestClassifier, ExtraTreesClassifier, AdaBoostClassifier

In [2]:
train = pd.read_csv('data/porto_train.csv')
test = pd.read_csv('data/porto_test.csv')

In [3]:
id_test = test['id'].values
target_train = train['target'].values

train = train.drop(['id', 'target'], axis = 1)
test = test.drop('id', axis = 1)

# 다른 kernel을 통해 ps_calc 변수가 유용하지 않다는걸 알아냄.
col_to_drop = train.columns[train.columns.str.startswith('ps_calc')]
train = train.drop(col_to_drop, axis = 1)
test = test.drop(col_to_drop, axis = 1)

# 다른 kernel을 통해 Missing value가 -1임을 알아냈음.
train = train.replace(-1, np.nan)
test = test.replace(-1, np.nan)

# Dummification
cat_features = [a for a in train.columns if a.endswith('cat')]

for column in cat_features:
    temp = pd.get_dummies(pd.Series(train[column]))
    train = pd.concat([train, temp], axis =1 )
    train = train.drop([column], axis =1)
    
for column in cat_features:
    temp = pd.get_dummies(pd.Series(test[column]))
    test = pd.concat([test, temp], axis =1)
    test = test.drop([column], axis = 1)

print(train.values.shape, test.values.shape)

(595212, 198) (892816, 198)


## Modeling

In [4]:
class Ensemble(object):
    def __init__(self, n_splits, stacker, base_models):
        self.n_splits = n_splits
        self.stacker = stacker
        self.base_models = base_models

    def fit_predict(self, X, y, T):
        X = np.array(X)
        y = np.array(y)
        T = np.array(T)

        folds = list(StratifiedKFold(n_splits=self.n_splits, shuffle=True, random_state=2016).split(X, y))

        S_train = np.zeros((X.shape[0], len(self.base_models)))
        S_test = np.zeros((T.shape[0], len(self.base_models)))
        
        for i, clf in enumerate(self.base_models):

            S_test_i = np.zeros((T.shape[0], self.n_splits))

            for j, (train_idx, test_idx) in enumerate(folds):
                X_train = X[train_idx]
                y_train = y[train_idx]
                X_holdout = X[test_idx]
#                y_holdout = y[test_idx]

                print ("Fit %s fold %d" % (str(clf).split('(')[0], j+1))
                clf.fit(X_train, y_train)
#                cross_score = cross_val_score(clf, X_train, y_train, cv=3, scoring='roc_auc')
#                print("    cross_score: %.5f" % (cross_score.mean()))
                y_pred = clf.predict_proba(X_holdout)[:,1]                

                S_train[test_idx, i] = y_pred
                S_test_i[:, j] = clf.predict_proba(T)[:,1]
            S_test[:, i] = S_test_i.mean(axis=1)

        results = cross_val_score(self.stacker, S_train, y, cv=3, scoring='roc_auc')
        print("Stacker score: %.5f" % (results.mean()))

        self.stacker.fit(S_train, y)
        res = self.stacker.predict_proba(S_test)[:,1]
        return res

In [5]:
# LightGBM params 1
lgb_params = {}
lgb_params['learning_rate'] = 0.02
lgb_params['n_estimators'] = 100
lgb_params['max_bin'] = 10
lgb_params['subsample'] = 0.8
lgb_params['subsample_freq'] = 10
lgb_params['colsample_bytree'] = 0.8
lgb_params['min_child_samples'] = 500
lgb_params['seed'] = 99

# LightGBM params2
lgb_params2 = {}
lgb_params2['n_estimators'] = 200
lgb_params2['learning_rate'] = 0.02
lgb_params2['colsample_bytree'] = 0.3
lgb_params2['subsample'] = 0.7
lgb_params2['subsample_freq'] = 2
lgb_params2['num_leaves'] = 16
lgb_params2['seed'] = 99

# LIghtGBM params3
lgb_params3 = {}
lgb_params3['n_estimators'] = 300
lgb_params3['max_depth'] = 4
lgb_params3['learning_rate'] = 0.02
lgb_params3['seed'] = 99

In [6]:
lgb_model = LGBMClassifier(**lgb_params)
lgb_model2 = LGBMClassifier(**lgb_params2)
lgb_model3 = LGBMClassifier(**lgb_params3)

log_model = LogisticRegression()

stack = Ensemble(n_splits = 3,
                stacker = log_model,
                base_models = (lgb_model, lgb_model2, lgb_model3))

y_pred = stack.fit_predict(train, target_train, test)

Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Fit LGBMClassifier fold 1
Fit LGBMClassifier fold 2
Fit LGBMClassifier fold 3
Stacker score: 0.63754
